In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("../input/portland-oregon-avg-rider-monthly-data/portland-oregon-average-monthly-.csv")

In [ ]:
df

In [ ]:
df.columns=["Month","average ridership"]

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df["average ridership"].unique()

In [ ]:
df = df.drop(df.index[df['average ridership'] == ' n=114'])

In [ ]:
df["average ridership"]=df["average ridership"].astype(np.int32)

In [ ]:
df

In [ ]:
df["Month"]=pd.to_datetime(df["Month"])

In [ ]:
df.dtypes

In [ ]:
pt=df.pivot_table(index="Month" , values="average ridership")
sns.heatmap(pt)

In [ ]:
df.set_index("Month",inplace=True)

In [ ]:
df

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
sd=seasonal_decompose(df["average ridership"])
sd.plot()

In [ ]:
df["average ridership"].plot()

In [ ]:
from statsmodels.tsa.stattools import adfuller as ad
add=ad(df["average ridership"])
print("ad statistic",add[0])
print("ad probability",add[1])
print("ad lags",add[2])
print("ad  total values",add[3])

In [ ]:
#differencing
df["firstdiff"]=df["average ridership"]-df["average ridership"].shift(1)
df

In [ ]:
ad(df["firstdiff"].dropna())

In [ ]:
df["seconddiff"]=df["firstdiff"]-df["firstdiff"].shift(1)

In [ ]:
ad(df["seconddiff"].dropna())

In [ ]:
#d=1

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_pacf(df["seconddiff"].dropna())


In [ ]:
plot_acf(df["seconddiff"].dropna())

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(df["average ridership"],order=(0,1,0),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:
print(results.summary())

In [ ]:
len(df)

In [ ]:
df.size

In [ ]:
df["predicted"]=results.predict(start=102,end=120,dynamic=True)

In [ ]:
df["predicted"]

In [ ]:
df[["average ridership","predicted"]].plot()

In [ ]:
len(df["average ridership"])

In [ ]:
from pandas.tseries.offsets import DateOffset as do
futuredates=[df.index[-1]+do(months=x) for x in range(0,10)]
dates=pd.DataFrame(index=futuredates,columns=df.columns)

In [ ]:
concatdf=pd.concat([df,dates])

In [ ]:
concatdf

In [ ]:
concatdf["True"]=results.predict(start=114,end=125,dynamic=True)

In [ ]:
concatdf["True"]

In [ ]:
concatdf[["average ridership","True"]].plot()

In [ ]:
concatdf.tail()

In [ ]:
concatdf.info()